# Original Data Cleaning

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [2]:
df=pd.read_csv("../data/A1c_Data_Requesta1c.csv")

## What does our data look like? 

When cleaning the data, we need to consider the following:
1. Data normalization: a process where data in a database is organized in a way that the user can utilize it to conduct analysis and answer questions. This process entails eliminating redundancy and logically group data together. This data was not normalized, there are two dataframes grouped together in one in a way that doesn't make sense (one responsible for a1c and when bloodwork was done, and the other holding information only on office visits). There are also many duplicates in the data. 
2. We have 273,485 rows of data and 8 columns. This data represents 1,099 patients. 
3. Each patient has multiple types of insurances listed, and we are unsure which is the most updated one or what many of them mean. For this reason, I don't believe we may have to eliminate this column for further analysis. 

In [3]:
df.head()

,PatientId,Age,Birthdate,VisitDate,SUMMARY,OBSDATE,Date of A1c,Insurance
0,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",7/21/2017 17:21:00,8.1,Dental Sliding Fee
1,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",7/21/2017 17:21:00,8.1,Sliding Fee Scale
2,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",1/19/2019 0:01:15,7.6,Dental Sliding Fee
3,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",1/19/2019 0:01:15,7.6,Sliding Fee Scale
4,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",2/8/2018 19:50:00,8.3,Dental Sliding Fee


How many patients do we have in this dataset? 

In [4]:
#Check number of patients by ID
df["PatientId"].value_counts()

115425    6072
971       5880
386       5247
111755    4422
201709    3410
          ... 
234272       2
241167       2
233853       1
234976       1
233208       1
Name: PatientId, Length: 1099, dtype: int64

In [5]:
#change column names so we can more easily work with columns
df.rename(columns={"PatientId": "patient_id","Age": "age", "VisitDate": "visit_date", "SUMMARY":"summary","OBSDATE":"obs_date","Date of A1c":"a1c","Insurance":"insurance"},inplace=True)

In [6]:
#check rows and columns 
df.shape

(273485, 8)

In [7]:
df["insurance"].value_counts()

Sliding Fee Scale                                63025
Medicaid - Husky D                               28659
Medicaid - Husky C                               24414
DO NOT USE - Medicaid State of CT                23468
National Government Services                     22005
Dental Sliding Fee                               17748
Medicaid - Husky A                               15570
Connecticare Exchange                             6944
DO NOT USE-Community Health Network - Husky A     6363
Medicaid State of CT                              6318
DO NOT USE - CHN Saga- Medicaid LIA               6174
Anthem Medicare                                   5834
United Healthcare Medicare                        4656
Connecticare                                      4546
CT Behavioral Health Partnership                  4216
Anthem BCBS                                       4010
Aetna Commercial                                  2675
Aetna Medicare -HMO                               2657
CT Dental 

In [8]:
#check how many null rows we have 
df.isnull().sum()

patient_id      0
age             0
Birthdate       0
visit_date      0
summary       218
obs_date        0
a1c            32
insurance       0
dtype: int64

In [9]:
#drop rows where a1c is missing since it is only 32 out of 273485
df.dropna(subset=["a1c"],inplace=True)

In [10]:
df.reset_index(inplace=True)

In [11]:
#check to make sure the null values were deleted
df.isnull().sum()

index           0
patient_id      0
age             0
Birthdate       0
visit_date      0
summary       218
obs_date        0
a1c             0
insurance       0
dtype: int64

In [12]:
#drop insurance and birthday 
df_no_ins= df.drop(columns=["insurance","Birthdate"])

In [13]:
#check datatypes: are the columns meant to be numerical, numerical? Are the dates in the dataframe datetime datatypes? 
df_no_ins.dtypes

index           int64
patient_id      int64
age             int64
visit_date     object
summary        object
obs_date       object
a1c           float64
dtype: object

In [15]:
#for objects that need to be converted into floats, none in the original dtype
# convert = lambda x: float(x)

In [16]:
#convert visit date and obs date to datetime, and take out time 
df_no_ins['visit_date'] = pd.to_datetime(df_no_ins['visit_date'])

In [17]:
df_no_ins['visit_date']= df_no_ins['visit_date'].dt.date

In [18]:
df_no_ins['visit_date'] = pd.to_datetime(df_no_ins['visit_date'])

In [19]:
df_no_ins['obs_date'] = pd.to_datetime(df_no_ins['obs_date'])

In [20]:
df_no_ins['obs_date'] = df_no_ins['obs_date'].dt.date

In [21]:
df_no_ins['obs_date'] = pd.to_datetime(df_no_ins['obs_date'])

In [22]:
df_no_ins.dtypes

index                  int64
patient_id             int64
age                    int64
visit_date    datetime64[ns]
summary               object
obs_date      datetime64[ns]
a1c                  float64
dtype: object

In [23]:
df_no_ins["a1c"].sort_values(ascending=True)

204598     0.089
204659     0.089
204658     0.089
204651     0.089
204650     0.089
           ...  
268928    19.700
268929    19.700
268930    19.700
268931    19.700
268927    19.700
Name: a1c, Length: 273453, dtype: float64

In [24]:
#.089 is too low
df_no_ins["a1c"].min()

0.08900000000000001

In [25]:
df_no_ins["a1c"].max()

19.7

In [26]:
#mean a1c value is 8.9
df_no_ins["a1c"].describe()

count    273453.000000
mean          8.902426
std           1.760240
min           0.089000
25%           7.500000
50%           8.400000
75%           9.900000
max          19.700000
Name: a1c, dtype: float64

In [27]:
df_no_ins["a1c"].where(df_no_ins["a1c"]!= 0.08900000000000001, 8.9,inplace=True)

In [30]:
df_no_ins.drop(columns=["index"],inplace=True)

In [32]:
df_no_ins["a1c"].min()

1.7

In [33]:
mask= df_no_ins.loc[df_no_ins["a1c"]== 1.7]

In [36]:
mask

,patient_id,age,visit_date,summary,obs_date,a1c
59052,115425,59,2018-07-06,INR f/u 2.5 on 8.5 mg qd,2019-09-13,1.7
59053,115425,59,2018-07-06,INR f/u 2.5 on 8.5 mg qd,2019-09-13,1.7
59054,115425,59,2018-07-06,INR f/u 2.5 on 8.5 mg qd,2019-09-13,1.7
59055,115425,59,2018-07-06,INR f/u 2.5 on 8.5 mg qd,2019-09-13,1.7
59056,115425,59,2018-07-06,INR f/u 2.5 on 8.5 mg qd,2019-09-13,1.7
...,...,...,...,...,...,...
65077,115425,59,2018-02-21,"hypotension, f/u DM type 2, COPD, OSA, HLD",2019-09-13,1.7
65078,115425,59,2018-02-21,"hypotension, f/u DM type 2, COPD, OSA, HLD",2019-09-13,1.7
65079,115425,59,2018-02-21,"hypotension, f/u DM type 2, COPD, OSA, HLD",2019-09-13,1.7
65080,115425,59,2018-02-21,"hypotension, f/u DM type 2, COPD, OSA, HLD",2019-09-13,1.7


In [35]:
mask.index

Int64Index([59052, 59053, 59054, 59055, 59056, 59057, 59070, 59071, 59072,
            59073,
            ...
            65066, 65067, 65068, 65069, 65076, 65077, 65078, 65079, 65080,
            65081],
           dtype='int64', length=552)

In [38]:
df_no_ins.shape

(273453, 6)

In [56]:
df_no_ins.drop(mask.index,axis=0,inplace=True)

In [57]:
df_no_ins["a1c"].min()

7.0

In [58]:
df_no_ins.shape

(272901, 6)

In [64]:
#time to drop duplicates!
df_no_ins.drop_duplicates(inplace=True)

In [65]:
df_no_ins.reset_index(drop=True,inplace=True)

In [66]:
df_no_ins.shape
# dropped 210,958 

(61943, 6)

Add column to dummy a1c level so we can use both classification and regression models 

In [67]:
df_no_ins["a1c_level"]=""

for i in range(df_no_ins.shape[0]):
    if df_no_ins["a1c"][i] <=8:
        df_no_ins["a1c_level"][i]= 1
    else: df_no_ins["a1c_level"][i]=0
        #1 is controlled
        #0 is uncontrolled 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [68]:
df_no_ins.head()

,patient_id,age,visit_date,summary,obs_date,a1c,a1c_level
0,72,56,2017-07-19,"followup diabetes, med refill",2017-07-21,8.1,0
1,72,56,2017-07-19,"followup diabetes, med refill",2019-01-19,7.6,1
2,72,56,2017-07-19,"followup diabetes, med refill",2018-02-08,8.3,0
3,72,56,2017-07-19,"followup diabetes, med refill",2017-10-20,7.7,1
4,72,56,2017-07-19,"followup diabetes, med refill",2019-06-08,7.1,1


In [70]:
df_no_ins.to_csv("../data/cleaned_data.csv")